# Final Data Capstone Project: Cafe Setup in Singapore

### Problem description and background

Singapore is known as a food haven, with many new eateries, restaurants and cafes springing up each year. However, the failure rate is also high. Some commonly cited statistics include: 

- Of the 369 cafes, coffee houses and snack bars set up in 2011, over half had shut down by 2014
- Of the 391 cafes, coffee houses and snack bars registered in 2013, approximately a quarter had called it quits by the end of 2014

A variety of reasons have been cited for these high failure rates, such as lack of experience, poor planning and a tight labour market. Location is another key factor that can affect business success.

### Description of the data and how it will be used

Using FourSquare data, we hope to provide a budding entrepreneur who is hoping to start a cafe in Singapore with recommendations on where to set up their cafe. To ensure that:

1. There will be adequate foot traffic (and in line with the urban decentralisation plan, e.g. https://www.population.sg/articles/singapores-urban-planning-in-five-points), 

2. Cafe is easily accessible

the focus will be on identifying which train stations will serve as the most suitable locations for setting up a cafe. Of particular interest is one of the newest train lines - the North-East (Purple) train line. This train line has 16 stations.

It is assumed that an ideal location for a cafe will be one that:

- Does not already have lots of cafes
- Has other eateries 

##### Train station dataset

This dataset was created by user hxchua and uploaded on the dataworld website (https://data.world/hxchua/train-stations-in-singapore). The list of train stations was compiled in June 2017. It contains 9 variables and the ones of primary interest for this project are:

- Station Name
- Latitude
- Longitude

An example of this would be: 

- Station Name: Boon Keng MRT Station NE9
- Latitude: 1.319396
- Longitude: 103.861679

##### FourSquare dataset

This dataset contains information on the top 10 most common venues for each of the train station locations, within a 300m radius.

Sticking with the example of Boon Keng MRT station NE9, the top 10 most common venues found are: Chinese Restaurant, Asian Restaurant, Bakery, Noodle House, Sandwich Place, Dessert Shop, Food Court, Soup Place, Fast Food Restaurant, Vegetarian/Vegan Restaurant



### Methodology

1. Convert the train station dataset from a CSV file to a Pandas dataframe
2. Retain only the necessary columns, i.e. station name, latitude and longitude (Note: Station name is a combination of the name of the train station as well as its station code)
3. Overlay the train station locations on a map of Singapore
4. Obtain FourSquare data (nearby venues - within a 300m radius) for these 16 train stations
5. Determine the top ten most common venues for each of these train stations
6. Evaluate what the most suitable locations would be for setting up a new cafe by identifying the train stations where cafes or coffee shops already exist, and ones where there are none
7. Determine what clusters exist among the train stations using the k-means clustering method and visualise the result on a map


### Results

In [1]:
#Import necessary libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  50.90 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.07 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  31.06 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  46.27 MB/s
Libraries imported.


In [2]:
# The code was removed by Watson Studio for sharing.

,OBJECTID,STN_NAME,STN_NO,X,Y,Latitude,Longitude,COLOR
0,12,ADMIRALTY MRT STATION,NS10,24402.1063,46918.1131,1.440585,103.800998,RED
1,16,ALJUNIED MRT STATION,EW9,33518.6049,33190.0020,1.316433,103.882893,GREEN
2,17,ANG MO KIO MRT STATION,NS16,29848.9836,39040.9896,1.369347,103.849928,RED
3,33,ANG MO KIO MRT STATION,NS16,29807.2655,39105.7720,1.369933,103.849553,RED
4,81,BAKAU LRT STATION,SE3,36026.0821,41113.8766,1.388093,103.905418,OTHERS


In [3]:
#Refining the train stations dataset so it only includes the full station name, latitude and longitude
NEstations = df[df.COLOR == 'PURPLE']
df1 = NEstations[['STN_NAME', 'STN_NO','Latitude', 'Longitude']]
df1['FullName']= df1['STN_NAME']+ ' ' + df1['STN_NO']
df1.head()
stations = df1[['FullName', 'Latitude', 'Longitude']]

#Replacing the coordinates of Outram Park station that are incorrect
stations.loc[117,'Latitude'] = 1.280225
stations.loc[117,'Longitude'] = 103.839486
stations
#stations.drop_duplicates(subset='FullName', keep='first', inplace=True)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,FullName,Latitude,Longitude
17,BOON KENG MRT STATION NE9,1.319396,103.861679
23,BUANGKOK MRT STATION NE15,1.382878,103.893104
37,CHINATOWN MRT STATION NE4,1.284360,103.843424
44,CLARKE QUAY MRT STATION NE5,1.288386,103.846552
53,DHOBY GHAUT MRT STATION NE6,1.299705,103.845485
63,FARRER PARK MRT STATION NE8,1.312360,103.854172
69,HARBOURFRONT MRT STATION NE1,1.265473,103.821446
74,HOUGANG MRT STATION NE14,1.371292,103.892364
89,KOVAN MRT STATION NE13,1.360179,103.885050
97,LITTLE INDIA MRT STATION NE7,1.307198,103.848581


In [4]:
#Obtaining the longitude and latitude of Singapore
address = 'Singapore, SG'

geolocator = Nominatim(user_agent="sg_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Singapore are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Singapore are 1.2904753, 103.8520359.


In [5]:
# mapping the train station locations in Singapore using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=11.3)

# add markers to map
for lat, lng, station in zip(stations['Latitude'], stations['Longitude'], stations['FullName']):
    label = '{}'.format(station)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sg)  
    
map_sg

In [6]:
# The code was removed by Watson Studio for sharing.

In [7]:
#defining a function to get nearby venues - within a 300m radius 

def getNearbyVenues(names, latitudes, longitudes, radius=300, LIMIT=20):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Station', 
                  'Station Latitude', 
                  'Station Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
station_venues = getNearbyVenues(names=stations['FullName'],
                                   latitudes=stations['Latitude'],
                                   longitudes=stations['Longitude']
                                  )

BOON KENG MRT STATION NE9
BUANGKOK MRT STATION NE15
CHINATOWN MRT STATION NE4
CLARKE QUAY MRT STATION NE5
DHOBY GHAUT MRT STATION NE6
FARRER PARK MRT STATION NE8
HARBOURFRONT MRT STATION NE1
HOUGANG MRT STATION NE14
KOVAN MRT STATION NE13
LITTLE INDIA MRT STATION NE7
OUTRAM PARK MRT STATION NE3
POTONG PASIR MRT STATION NE10
PUNGGOL MRT STATION NE17
SENGKANG MRT STATION NE16
SERANGOON MRT STATION NE12
WOODLEIGH MRT STATION NE11


In [9]:
#Determining the number of venues per train station
station_venues.groupby('Station').count()

,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Station,,,,,,
BOON KENG MRT STATION NE9,15,15,15,15,15,15
BUANGKOK MRT STATION NE15,7,7,7,7,7,7
CHINATOWN MRT STATION NE4,20,20,20,20,20,20
CLARKE QUAY MRT STATION NE5,20,20,20,20,20,20
DHOBY GHAUT MRT STATION NE6,20,20,20,20,20,20
FARRER PARK MRT STATION NE8,20,20,20,20,20,20
HARBOURFRONT MRT STATION NE1,20,20,20,20,20,20
HOUGANG MRT STATION NE14,20,20,20,20,20,20
KOVAN MRT STATION NE13,20,20,20,20,20,20


In [10]:
#Determining the number of unique venue categories across the 16 train station locations
print('There are {} unique categories.'.format(len(station_venues['Venue Category'].unique())))

There are 110 unique categories.


In [11]:
# one hot encoding
station_onehot = pd.get_dummies(station_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
station_onehot['Station'] = station_venues['Station'] 

# move neighborhood column to the first column
fixed_columns = [station_onehot.columns[-1]] + list(station_onehot.columns[:-1])
station_onehot = station_onehot[fixed_columns]

station_onehot.head()

,Station,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Bus Line,Bus Station,Café,Cantonese Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Department Store,Dessert Shop,Discount Store,Dumpling Restaurant,English Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food Court,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gastropub,General Entertainment,German Restaurant,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lounge,Market,Metro Station,Mexican Restaurant,Miscellaneous Shop,Movie Theater,Multiplex,Museum,Nail Salon,Neighborhood,Noodle House,North Indian Restaurant,Pakistani Restaurant,Park,Pet Store,Pharmacy,Plaza,Portuguese Restaurant,Restaurant,River,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Snack Place,Soccer Field,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Supermarket,Sushi Restaurant,Swiss Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,BOON KENG MRT STATION NE9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,BOON KENG MRT STATION NE9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,BOON KENG MRT STATION NE9,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,BOON KENG MRT STATION NE9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,BOON KENG MRT STATION NE9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
station_grouped = station_onehot.groupby('Station').mean().reset_index()
station_grouped

,Station,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Bus Line,Bus Station,Café,Cantonese Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Department Store,Dessert Shop,Discount Store,Dumpling Restaurant,English Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food Court,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gastropub,General Entertainment,German Restaurant,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lounge,Market,Metro Station,Mexican Restaurant,Miscellaneous Shop,Movie Theater,Multiplex,Museum,Nail Salon,Neighborhood,Noodle House,North Indian Restaurant,Pakistani Restaurant,Park,Pet Store,Pharmacy,Plaza,Portuguese Restaurant,Restaurant,River,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Snack Place,Soccer Field,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Supermarket,Sushi Restaurant,Swiss Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,BOON KENG MRT STATION NE9,0.00,0.133333,0.0,0.133333,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.20,0.00,0.00,0.000000,0.0,0.00,0.00,0.066667,0.00,0.00,0.00,0.066667,0.00,0.00,0.00,0.066667,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.133333,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.066667,0.00,0.00,0.00,0.00,0.00,0.00,0.066667,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.066667,0.00,0.00,0.00,0.00,0.000000,0.00
1,BUANGKOK MRT STATION NE15,0.00,0.000000,0.0,0.142857,0.00,0.285714,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.0,0.00,0.00,0.000000,0.00,0.00,0.00,0.142857,0.00,0.00,0.00,0.142857,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.000000,0.142857,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.142857,0.00
2,CHINATOWN MRT STATION NE4,0.00,0.000000,0.0,0.000000,0.00,0.000000,0.05,0.05,0.100000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.15,0.00,0.00,0.000000,0.0,0.00,0.00,0.050000,0.00,0.05,0.00,0.000000,0.00,0.05,0.00,0.050000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.05,0.15,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.10,0.00,0.00,0.00,0.00,0.000000,0.1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00
3,CLARKE QUAY MRT STATION NE5,0.00,0.050000,0.0,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.05,0.00,0.00,0.000000,0.05,0.00,0.00,0.000000,0.000000,0.05,0.000000,0.00,0.00,0.05,0.000000,0.0,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.10,0.05,0.00,0.000000,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.1,0.

In [13]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Station']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
station_venues_sorted = pd.DataFrame(columns=columns)
station_venues_sorted['Station'] = station_grouped['Station']

for ind in np.arange(station_grouped.shape[0]):
    station_venues_sorted.iloc[ind, 1:] = return_most_common_venues(station_grouped.iloc[ind, :], num_top_venues)

station_venues_sorted.head()

,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,BOON KENG MRT STATION NE9,Chinese Restaurant,Asian Restaurant,Bakery,Noodle House,Sandwich Place,Dessert Shop,Food Court,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Soup Place
1,BUANGKOK MRT STATION NE15,Basketball Court,Wings Joint,Fast Food Restaurant,Bakery,Food Court,Metro Station,German Restaurant,English Restaurant,Fish & Chips Shop,Flea Market
2,CHINATOWN MRT STATION NE4,Hostel,Chinese Restaurant,Beer Garden,Seafood Restaurant,Spa,History Museum,Bed & Breakfast,Beer Bar,Pharmacy,Flea Market
3,CLARKE QUAY MRT STATION NE5,Hotel,Lounge,Seafood Restaurant,Yoga Studio,Spanish Restaurant,Asian Restaurant,Bistro,Bubble Tea Shop,Café,Cocktail Bar
4,DHOBY GHAUT MRT STATION NE6,Yoga Studio,Soup Place,Park,Movie Theater,Miscellaneous Shop,Indonesian Restaurant,Hotpot Restaurant,Gym,Gift Shop,General Entertainment


In [36]:
# set number of clusters
kclusters = 5

station_grouped_clustering = station_grouped.drop('Station', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(station_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 1, 1, 1, 1, 1, 1, 0, 4], dtype=int32)

In [37]:
# add clustering labels
station_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

station_merged = stations
station_merged.rename(columns={'FullName':'Station'}, inplace=True)

# merge station_grouped with station_data to add latitude/longitude for each neighborhood
station_merged = station_merged.join(station_venues_sorted.set_index('Station'), on='Station')

station_merged.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,Station,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,BOON KENG MRT STATION NE9,1.319396,103.861679,0,Chinese Restaurant,Asian Restaurant,Bakery,Noodle House,Sandwich Place,Dessert Shop,Food Court,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Soup Place
23,BUANGKOK MRT STATION NE15,1.382878,103.893104,2,Basketball Court,Wings Joint,Fast Food Restaurant,Bakery,Food Court,Metro Station,German Restaurant,English Restaurant,Fish & Chips Shop,Flea Market
37,CHINATOWN MRT STATION NE4,1.284360,103.843424,1,Hostel,Chinese Restaurant,Beer Garden,Seafood Restaurant,Spa,History Museum,Bed & Breakfast,Beer Bar,Pharmacy,Flea Market
44,CLARKE QUAY MRT STATION NE5,1.288386,103.846552,1,Hotel,Lounge,Seafood Restaurant,Yoga Studio,Spanish Restaurant,Asian Restaurant,Bistro,Bubble Tea Shop,Café,Cocktail Bar
53,DHOBY GHAUT MRT STATION NE6,1.299705,103.845485,1,Yoga Studio,Soup Place,Park,Movie Theater,Miscellaneous Shop,Indonesian Restaurant,Hotpot Restaurant,Gym,Gift Shop,General Entertainment


In [38]:
hits = {'Café', 'Coffee Shop'}
eateries = {'Asian Restaurant', 'Fast Food Restaurant', 'Indonesian Restaurant', 'Beer Bar', 'Chinese Restaurant', 'Seafood Restaurant', 'Soup Place', 'Pakistani Restaurant', 'Australian Restaurant', 'Indian Restaurant', 'Salad Place', 'Swiss Restaurant', 'Japanese Restaurant', 'Burger Joint', 'Korean Restaurant', 'Tapas Restaurant', 'Italian Restaurant', 'Cantonese Restaurant', 'Thai Restaurant', 'Portuguese Restaurant', 'Sushi Restaurant', 'Vegetarian / Vegan Restaurant', 'German Restaurant', 'English Restaurant', 'Food Court', 'Café', 'Coffee Shop', 'Fish & Chips Shop', 'Sandwich Place', 'Frozen Yogurt Shop', 'Dessert Shop', 'Gastropub', 'Wings Joint', 'Bubble Tea Shop', 'Ice Cream Shop', 'Beer Garden', 'Noodle House', 'Spanish Restaurant', 'Hotpot Restaurant', 'Snack Place', 'Bistro', 'Breakfast Spot'}

station_merged['Number of Coffee Places'] = station_merged.isin(hits).sum(1)
station_merged['Number of Eateries'] = station_merged.isin(eateries).sum(1)
station_merged.sort_values(['Number of Coffee Places', 'Number of Eateries'], ascending=False)

,Station,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Number of Coffee Places,Number of Eateries
89,KOVAN MRT STATION NE13,1.360179,103.885050,0,Café,Asian Restaurant,Chinese Restaurant,Noodle House,Coffee Shop,Korean Restaurant,Soccer Field,Building,Snack Place,Food Court,2,8
74,HOUGANG MRT STATION NE14,1.371292,103.892364,1,Supermarket,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Food Court,Bookstore,Fish & Chips Shop,Shopping Mall,Café,Frozen Yogurt Shop,2,7
147,SENGKANG MRT STATION NE16,1.391695,103.895466,1,Fast Food Restaurant,Café,Food Court,Coffee Shop,Metro Station,Restaurant,English Restaurant,Sandwich Place,Chinese Restaurant,Shopping Mall,2,7
117,OUTRAM PARK MRT STATION NE3,1.280225,103.839486,1,Japanese Restaurant,Australian Restaurant,Chinese Restaurant,Café,Tapas Restaurant,Dessert Shop,Cocktail Bar,Seafood Restaurant,Nail Salon,Burger Joint,1,8
131,PUNGGOL MRT STATION NE17,1.404547,103.902052,1,Chinese Restaurant,Japanese Restaurant,Sushi Restaurant,Food Court,Multiplex,Fast Food Restaurant,Discount Store,Coffee Shop,Ice Cream Shop,Sandwich Place,1,8
44,CLARKE QUAY MRT STATION NE5,1.288386,103.846552,1,Hotel,Lounge,Seafood Restaurant,Yoga Studio,Spanish Restaurant,Asian Restaurant,Bistro,Bubble Tea Shop,Café,Cocktail Bar,1,6
97,LITTLE INDIA MRT STATION NE7,1.307198,103.848581,4,Indian Restaurant,Fruit & Vegetable Store,Cantonese Restaurant,Breakfast Spot,Market,Museum,Thai Restaurant,Beer Garden,Neighborhood,Coffee Shop,1,6
63,FARRER PARK MRT STATION NE8,1.312360,103.854172,1,Hotel,Café,Indian Restaurant,Chinese Restaurant,Clothing Store,Sporting Goods Shop,Dumpling Restaurant,Bubble Tea Shop,Food Court,Shopping Mall,1,5
127,POTONG PASIR MRT STATION NE10,1.331380,103.869046,1,Noodle House,Food Court,Snack Place,Fast Food Restaurant,Grocery Store,Convenience Store,Gym / Fitness Center,Market,Flower Shop,Coffee Shop,1,5
17,BOON KENG MRT STATION NE9,1.319396,103.861679,0,Chinese Restaurant,Asian Restaurant,Bakery,Noodle House,Sandwich Place,Dessert Shop,Food Court,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Soup Place,0,9


In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(station_merged['Latitude'], station_merged['Longitude'], station_merged['Station'], station_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Discussion

The table reflects that the majority of train stations on the North-East line already have at least one cafe or coffee shop. In particular, Kovan, Hougang and Sengkang stations have both cafes and coffee shops amongst their top 10 most common venues. Hence, it might be best to avoid setting up a new cafe at these stations, where there are established cafes.

Boon Keng, Serangoon and Buangkok stand out as train stations with a good number of eateries (between 6 to 9) amongst its top 10 most common venues, where there is no mention of cafes or coffee shops. These are likely to be the most promising areas for setting up a new cafe.

### Conclusion

Focussing on the just the North-East train line, FourSquare data suggests that the most promising areas for setting up a new cafe would be the Boon Keng,  Serangoon and Buangkok train stations. There are a good number of existing eateries at these stations, but cafes and coffee shops do not currently feature amongst their top 10 most common venues. 

### Link to presentation